In [1]:
# Importações básicas
import numpy as np # Para manipulação de arrays e cálculos numéricos eficientes
import pandas as pd # Para carregar, maipular e analisar conjutos de dados estruturados (DataFrames e Séries).
import matplotlib.pyplot as plt # Para criar gráficos e visualizar dados.
from sklearn import datasets, metrics # Contém conjuntos de dados prontos para teste (ex: iris, digits, wine, etc.). E a metrics fornece mpetricas para avaliar modelos, como precisão e modelo 
from sklearn.model_selection import train_test_split # Divide os dados em conjuto de treino e teste, essencial para avaliar modelos de ML

In [2]:
# Carregar o conjunto de dados iris
iris = datasets.load_iris() # Carrega esse conjunto de dados.
X = iris.data # Atribui as características (comprimento e largura das pétalas e sépalas) á variável "x"
y = iris.target # Atribui os rótulos das classes (0, 1 e 2, representando os três tipos de flores) á variável "y"

# Verificar as dimensões
print(f"Formato dos dados: {X.shape}") # mostra a dimensão da matriz de dados (número de amostras x número de características)
print(f"Formato dos rótulos: {y.shape}") # mostra a dimensão do vetor de rótulos, que deve ser (150), pois há 150 rótulos, um para cada amostra

# Dividir em conjuntos de treino e teste
#Divide os dados em dois conjuntos:
 #Treino (X_train, y_train) -> Usado para treinar o modelo (75% dos dados).
 #Teste (X_test, y_test) -> Usado para avaliar o modelo (25% dos dados)
#test_size=0.25 -> Define que 25% dos dados serão usados para teste.
#random_state=42 -> Garante que divisão será sempre a mesma quando o código for executado novamente (para reprodutibilidade)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

Formato dos dados: (150, 4)
Formato dos rótulos: (150,)


In [3]:
# StandardScaler -> Normaliza os dados, deixando-os com média 0 e desvio padrão 1.
# OneHotEnconder -> Transforma variáveis categóricas em valores numéricos (não usado diretamente nesse código, mas útil para categorizar dados)
# SimpleImputer -> Preenche valores ausentes com alguma estratégia (como a mediana ou a média)
# ColumnTransformer -> Aplica transformações diferentes a colunas específicas (útil quando temos variaveis numéricas e categóricas)
# Pipeline -> Cria um fluxo de pré-processamento automático
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Exemplo de escalonamento de features
scaler = StandardScaler() # calcula a média e o desvio padrão dos dados
X_train_scaled = scaler.fit_transform(X_train) # Ajusta os dados de treino e os transforma
X_test_scaled = scaler.transform(X_test) # Usa os mesmos parâmetros para transformar os dados de teste (sem recalcular a média  eo desvio padrão)

# Exemplo mais complexo com diferentes tipos de dados
# (normalmente usado quando temos features numéricas e categóricas)
numeric_features = [0, 1, 2, 3]  # índices das colunas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # preenche valores ausentes com a mediana
    ('scaler', StandardScaler()) # Escalona os valores para manter média 0 e desvio padrão 1
])

preprocessor = ColumnTransformer( # Aplica diferentes transformações a diferentes tipos de colunas
    transformers=[
        ('num', numeric_transformer, numeric_features) # Neste caso, aplica "numeric_transformer" ás colunas numéricas [0, 1, 2, 3]
    ])

    # Resumo
    # O codigo faz pré-processamento de dados para Machine Learning:
    # 1. Escalonamento: Normaliza os dados numéricos para evitar que algumas variaveis tenham mais peso que outras.
    # 2. Tratamento de valores ausentes> Preenche valores faltantes com a mediana.
    # 3. Pipeline: Encadeia os passos de transformação  para facilitar a aplicação em novos dados.

In [ ]:
from sklearn.linear_model import LogisticRegression # Modelo de regressao logística, usado para clsassificação binaria
from sklearn.ensemble import RandomForestClassifier # Modelo baseado em múltiplas árvores de decisão para classificação
from sklearn.svm import SVC # Modelo de Support Vector Machine (Máquina de Vetores de Suporte), usado para classificação

# Pipeline simples com pré-processamento e modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), # Aplica o pré-processamento definido antes (ColumnTransfromer com Standardscaler e SimpleImputer)
    ('classifier', RandomForestClassifier(random_state=42)) # Modelo de classificação. Neste caso, um RandomForestClassifier (floresta aleatória), que usa várias árvores de decisão para melhorar a precisão. e o random_state=42: define um valor fixo para que os resultados sejam reproduzíveis.
])

# Treinar o pipeline
# Primeiro, o preprocessor transforma os dados de treino
# Depois, o RandomForestClassifier aprende padrões nos dados transofrmados.
pipeline.fit(X_train, y_train)

# Fazer previsões
# Aplica o pré-processamneto e modelo treinado ao conjunto de teste
# Retorna as previsões "y_pred", que são as clasess previstas para os dados de teste
y_pred = pipeline.predict(X_test)

# Resumo
# 1. Pré-precesa os dados (preenchimento de valores ausentes e escalonamento)
# 2. treina um modelo de classificação (Random Forest)
# 3. faz previsões nos dados de teste

In [ ]:
# Avaliar a precisão
accuracy = metrics.accuracy_score(y_test, y_pred) # Calcula a acurácia, ou seja, a fração de previsões corretas sobre o total
print(f"Acurácia: {accuracy:.4f}")  # O "print" exibe a acurácia formatada com quatro casas decimais

# Relatório de classificação
print("Relatório de classificação:")
print(metrics.classification_report(y_test, y_pred,
                                   target_names=iris.target_names))

# Matriz de confusão
cm = metrics.confusion_matrix(y_test, y_pred)
print("Matriz de confusão:")
print(cm)

# Visualizar a matriz de confusão
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=iris.target_names,
           yticklabels=iris.target_names)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir parâmetros para busca
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30]
}

# Criar objeto de busca em grade
grid_search = GridSearchCV(
    pipeline, param_grid, cv=5, scoring='accuracy'
)

# Executar busca
grid_search.fit(X_train, y_train)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação:", grid_search.best_score_)

# Usar o melhor modelo
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(f"Acurácia com o melhor modelo: {metrics.accuracy_score(y_test, y_pred_best):.4f}")

In [ ]:
import joblib

# Salvar o modelo
joblib.dump(best_model, 'modelo_iris.pkl')

# Carregar o modelo
modelo_carregado = joblib.load('modelo_iris.pkl')

# Verificar se o modelo carregado funciona
y_pred_loaded = modelo_carregado.predict(X_test)
print(f"Acurácia do modelo carregado: {metrics.accuracy_score(y_test, y_pred_loaded):.4f}")

In [ ]:
# Carregar o conjunto de dados de câncer de mama
cancer = datasets.load_breast_cancer()
X = cancer.data
y = cancer.target

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Criar pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Definir parâmetros para busca
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'lbfgs']
}

# Busca em grade
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Melhor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Avaliação
print(f"Acurácia: {metrics.accuracy_score(y_test, y_pred):.4f}")
print("\nRelatório de classificação:")
print(metrics.classification_report(y_test, y_pred))